In [ ]:
# import
import numpy as np
import pandas as pd
import os.path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score
import os
import math
# label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================

def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type   

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            if label != "":
                dataset.append(
                                {   
                                    'uuid': root.split("\\")[-1],
                                    'label': label,
                                    'image_path': os.path.join(root, file)
                                }
                            )

    return dataset

def multi_input_generator(front_gen, side_gen):
    while True:
        front_batch, y1 = next(front_gen)
        side_batch, y2 = next(side_gen)
        assert (y1 == y2).all(), "Label mismatch!"  # 確保標籤一致
        yield ([front_batch, side_batch], y1)

def create_front_extract():
    pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    pretrained_model = pretrained_model_chosen(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    pretrained_model._name = 'AP_pretrain_resnet_model'
    pretrained_model.trainable = False
    return pretrained_model

def create_side_extract():
    pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    pretrained_model = pretrained_model_chosen(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg')
    pretrained_model._name = 'Lateral_pretrain_efficientnet_model'
    pretrained_model.trainable = False
    return pretrained_model

In [ ]:
## 參數設置
image_dir = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\front"
concat_type = "concat2"
class_count = 3

In [ ]:
## load data and  labels
# =========================
data = load_path(image_dir, class_count)
labels = []
filepaths = []
for row in data:
    labels.append(row['label'])
    filepaths.append(row['image_path'])

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([filepaths, labels], axis=1)
# =========================


In [ ]:
## split image
# =========================
train_df_front, test_df_front = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])
print("Training set label distribution:\n", train_df_front['Label'].value_counts(normalize=False))
print("Test set label distribution:\n", test_df_front['Label'].value_counts(normalize=False))
# =========================

preprocessing_function_chosen_front = tf.keras.applications.resnet50.preprocess_input
# preprocessing_function_chosen_side = tf.keras.applications.efficientnet.preprocess_input
preprocessing_function_chosen_side = tf.keras.applications.resnet50.preprocess_input

# front images
# =========================
train_generator_front = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                    preprocessing_function=preprocessing_function_chosen_front,
                                                                    validation_split=0.2)
test_generator_front = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen_front)

train_images_front = train_generator_front.flow_from_dataframe(
    dataframe=train_df_front,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False,
    seed=42,
    subset='training'
)

val_images_front = train_generator_front.flow_from_dataframe(
    dataframe=train_df_front,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False,
    seed=42,
    subset='validation'
)

test_images_front = test_generator_front.flow_from_dataframe(
    dataframe=test_df_front,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)
# =========================


# side images
# =========================
train_df_side = train_df_front.copy()
test_df_side = test_df_front.copy()
train_df_side.loc[:, "Filepath"] = train_df_front["Filepath"].str.replace("front", "side")
test_df_side.loc[:, "Filepath"] = test_df_side["Filepath"].str.replace("front", "side")

train_generator_side = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,
                                                                    preprocessing_function=preprocessing_function_chosen_side,
                                                                    validation_split=0.2)
test_generator_side = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen_side)

train_images_side = train_generator_side.flow_from_dataframe(
    dataframe=train_df_side,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False,
    seed=42,
    subset='training'
)

val_images_side = train_generator_side.flow_from_dataframe(
    dataframe=train_df_side,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False,
    seed=42,
    subset='validation'
)

test_images_side = test_generator_side.flow_from_dataframe(
    dataframe=test_df_side,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)
# =========================


In [ ]:
# test_df_side
# test_df_front


In [ ]:
from tensorflow.keras.layers import Lambda
import random
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from deap import base, creator, tools, algorithms
import math
train_generator = multi_input_generator(train_images_front, train_images_side)
val_generator = multi_input_generator(val_images_front, val_images_side)

def create_model(gene_mask):
    input_front = Input(shape=(224, 224, 3), name="AP(Mortise)")
    input_side = Input(shape=(224, 224, 3), name="Lateral")

    # 特徵提取
    model_front = create_front_extract()
    features_front = model_front(input_front)
    features_front_x = Dense(128, activation='relu')(features_front)
    features_front_x = Dense(50, activation='relu')(features_front_x)

    model_side = create_side_extract()
    features_side = model_side(input_side)
    features_side_x = Dense(128, activation='relu')(features_side)
    features_side_x = Dense(50, activation='relu')(features_side_x)

    fused_features = Concatenate()([features_front_x, features_side_x])

    # 使用基因選擇特徵
    def apply_mask(tensor):
        return tensor * tf.constant(gene_mask, dtype=tf.float32)  # 只保留基因為 1 的特徵

    selected_features = Lambda(apply_mask)(fused_features)
    
    # selected_features = Dropout(0.1)(selected_features)
    final_output = Dense(class_count, activation='sigmoid', name='output_layer')(selected_features)

    model = Model(inputs=[input_front, input_side], outputs=final_output)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## early stop 
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

score = []
def evaluate_individual(individual):
    global score
    model = create_model(individual)  # 這裡的 individual 就是 100 維的 0/1 向量
    batch_size = 64

    ## early stop 
    history = model.fit(train_generator, validation_data=val_generator, callbacks=[early_stopping], epochs=30,
                steps_per_epoch= math.ceil(train_images_front.samples / batch_size), 
                validation_steps= math.ceil(val_images_front.samples / batch_size),
                verbose=1)
    accuracy=max(history.history['val_accuracy'])

    ## ==================
    # history = model.fit(train_generator, validation_data=val_generator, epochs=30, 
    #                     steps_per_epoch=math.ceil(train_images_front.samples / batch_size), 
    #                     validation_steps=math.ceil(val_images_front.samples / batch_size), 
    #                     verbose=0)
    # accuracy = history.history['val_accuracy'][-1]
    ## ==================

    # print("val_accuracy = ", accuracy)

    ## print test results
    # =========================
    test_generator = multi_input_generator(test_images_front, test_images_side)

    batch_size=32
    pred = model.predict(test_generator,  steps=math.ceil(test_images_front.samples / batch_size))
    predicted_labels = np.argmax(pred, axis=1)
    acc = accuracy_score(test_images_front.labels, predicted_labels)
    f1 = f1_score(test_images_front.labels, predicted_labels, average='macro')
    # print("test_accuracy = ", acc)
    # print("test_f1 = ", f1)
    score.append([np.round(accuracy,2), np.round(acc,2), np.round(f1,2)])
    print("-")
    # =========================
    return accuracy,  # 必須返回元組

tmp_best = []
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # 這表示我們是要最大化目標 (比如最大化準確度)
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
# 生成長度為 100 的 0/1 基因序列
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)  # 交叉
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)  # 突變，每個基因有 10% 機率翻轉
toolbox.register("select", tools.selTournament, tournsize=3)  # 選擇
toolbox.register("evaluate", evaluate_individual)  # 評估

population_size = 50
num_generations = 10

population = toolbox.population(n=population_size)

for generation in range(num_generations):
    print(generation)
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.7:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < 0.2:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    invalid_individuals = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = list(map(toolbox.evaluate, invalid_individuals))

    for ind, fit in zip(invalid_individuals, fitnesses):
        ind.fitness.values = fit

    population[:] = offspring
    best_individual = tools.selBest(population, 1)[0]
    tmp_best.append(best_individual)
    # print(evaluate_individual(best_individual)) 
    print(score[-1])
    print(f"目前最佳特徵選擇: {best_individual}")
    print("===========================================================================")





In [ ]:
# best_index = next(i for i, ind in enumerate(population) if ind == best_individual)

In [ ]:
best_individuals = tools.selBest(population, 5)  # 選擇前 5 個最好的個體
for i, individual in enumerate(best_individuals):
    print(f"Evaluating Feature Subset {i+1}: {individual}")

In [ ]:
# # 使用最佳特徵選擇重新創建模型
# best_model = create_model(best_individual)

# # 訓練最終模型
# batch_size = 64
# history = best_model.fit(train_generator, validation_data=val_generator, epochs=5, 
#                          steps_per_epoch=math.ceil(train_images_front.samples / batch_size), 
#                          validation_steps=math.ceil(val_images_front.samples / batch_size), 
#                          verbose=0)

# # 存儲最佳模型
# best_model.save("best_feature_selection_model.h5")
# print("最佳模型已保存為 best_feature_selection_model.h5")
